**Kahou et al., "EmoNets: Multimodal deep learning approaches for emotion recognition in video," Journal on Multimodal User Interfaces, 2016.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**With Duplicate**

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv3D, MaxPooling3D, Flatten, Dense, LSTM, Dropout,
                                   Concatenate, BatchNormalization, Activation)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical

# 1. Data Loading & Duplication
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)

def duplicate_dataset(data, num_duplicates):
    duplicated_data = data.tolist()
    for _ in range(num_duplicates):
        duplicated_data.extend(data.tolist())
    return np.array(duplicated_data)

num_duplicates = 2
data = duplicate_dataset(data, num_duplicates)

# 2. Feature Extraction
X_eeg   = np.array([entry['eeg_data'] for entry in data])    # [samples, 5, 512]
X_video = np.array([entry['video_data'] for entry in data])  # [samples, 32, 64, 64]
X_ecg   = np.array([entry['ecg_data'] for entry in data])    # [samples, 2, 512]
y       = np.array([entry['labels'] for entry in data])

print(f"Number of samples after duplication: {len(data)}")

# 3. Video Padding (if needed)
if X_video.shape[1:] != (32, 64, 64):
    padded_video = []
    for video in X_video:
        pad_shape = [
            (0, max(0, 32 - video.shape[0])),
            (0, max(0, 64 - video.shape[1])),
            (0, max(0, 64 - video.shape[2]))
        ]
        padded_video.append(np.pad(video, pad_shape, mode='constant'))
    X_video = np.array(padded_video)

# 4. Balance Classes (both valence and arousal)
def balance_classes(X1, X2, X3, y, class_idx):
    labels = y[:, class_idx]
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    max_count = np.max(class_counts)

    balanced_X1, balanced_X2, balanced_X3, balanced_y = [], [], [], []
    for cls in unique_classes:
        cls_mask = labels == cls
        cls_X1, cls_X2, cls_X3, cls_y = X1[cls_mask], X2[cls_mask], X3[cls_mask], y[cls_mask]
        if len(cls_y) < max_count:
            X1_oversampled, X2_oversampled, X3_oversampled, y_oversampled = resample(
                cls_X1, cls_X2, cls_X3, cls_y,
                replace=True,
                n_samples=max_count - len(cls_y),
                random_state=42
            )
            cls_X1 = np.concatenate([cls_X1, X1_oversampled])
            cls_X2 = np.concatenate([cls_X2, X2_oversampled])
            cls_X3 = np.concatenate([cls_X3, X3_oversampled])
            cls_y = np.concatenate([cls_y, y_oversampled])
        balanced_X1.append(cls_X1)
        balanced_X2.append(cls_X2)
        balanced_X3.append(cls_X3)
        balanced_y.append(cls_y)
    return (np.concatenate(balanced_X1),
            np.concatenate(balanced_X2),
            np.concatenate(balanced_X3),
            np.concatenate(balanced_y))

# Balance by valence (index 1) and arousal (index 2)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=1)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=2)

# 5. Train/Test Split
X_eeg_train, X_eeg_test, X_video_train, X_video_test, X_ecg_train, X_ecg_test, y_train, y_test = train_test_split(
    X_eeg, X_video, X_ecg, y, test_size=0.2, random_state=42
)

num_valence_classes = int(np.max(y[:, 1])) + 1
num_arousal_classes = int(np.max(y[:, 2])) + 1

y_valence_train = to_categorical(y_train[:, 1], num_classes=num_valence_classes)
y_arousal_train = to_categorical(y_train[:, 2], num_classes=num_arousal_classes)
y_valence_test = to_categorical(y_test[:, 1], num_classes=num_valence_classes)
y_arousal_test = to_categorical(y_test[:, 2], num_classes=num_arousal_classes)

# 6. Model Definition
# 6a. State-of-the-Art 3D CNN Video Branch (Kahou et al. style)
video_input = Input(shape=(32, 64, 64, 1), name="video_input")
x = Conv3D(32, (3,3,3), strides=1, padding='same')(video_input)
x = Activation('relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv3D(64, (3,3,3), strides=1, padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
cnn_output = Dense(128, activation='relu')(x)

# 6b. EEG Branch (LSTM)
eeg_input = Input(shape=(5, 512), name="eeg_input")
lstm_eeg = LSTM(128, return_sequences=True)(eeg_input)
lstm_eeg = Dropout(0.2)(lstm_eeg)
lstm_eeg = LSTM(64)(lstm_eeg)
lstm_eeg_output = Dense(128, activation='relu')(lstm_eeg)

# 6c. ECG Branch (LSTM)
ecg_input = Input(shape=(2, 512), name="ecg_input")
lstm_ecg = LSTM(64, return_sequences=True)(ecg_input)
lstm_ecg = Dropout(0.2)(lstm_ecg)
lstm_ecg = LSTM(32)(lstm_ecg)
lstm_ecg_output = Dense(64, activation='relu')(lstm_ecg)

# 6d. Fusion & Output
combined = Concatenate()([cnn_output, lstm_eeg_output, lstm_ecg_output])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.2)(combined)
valence_output = Dense(num_valence_classes, activation='softmax', name="valence_output")(combined)
arousal_output = Dense(num_arousal_classes, activation='softmax', name="arousal_output")(combined)

model = Model(
    inputs=[video_input, eeg_input, ecg_input],
    outputs=[valence_output, arousal_output]
)

model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    metrics=['accuracy','accuracy']
)

model.summary()

# 7. Data Reshaping for Video Channel
X_video_train_input = X_video_train[..., np.newaxis] if X_video_train.ndim == 4 else X_video_train
X_video_test_input = X_video_test[..., np.newaxis] if X_video_test.ndim == 4 else X_video_test

# 8. Model Training
history = model.fit(
    [X_video_train_input, X_eeg_train, X_ecg_train],
    [y_valence_train, y_arousal_train],
    validation_data=(
        [X_video_test_input, X_eeg_test, X_ecg_test],
        [y_valence_test, y_arousal_test]
    ),
    epochs=50,
    batch_size=128
)

# 9. Model Evaluation
y_valence_pred, y_arousal_pred = model.predict([X_video_test_input, X_eeg_test, X_ecg_test])
y_valence_pred_classes = np.argmax(y_valence_pred, axis=1)
y_arousal_pred_classes = np.argmax(y_arousal_pred, axis=1)
y_valence_true = np.argmax(y_valence_test, axis=1)
y_arousal_true = np.argmax(y_arousal_test, axis=1)

valence_accuracy = accuracy_score(y_valence_true, y_valence_pred_classes)
arousal_accuracy = accuracy_score(y_arousal_true, y_arousal_pred_classes)

print(f"Valence Accuracy: {valence_accuracy * 100:.2f}%")
print(f"Arousal Accuracy: {arousal_accuracy * 100:.2f}%")


Number of samples after duplication: 558


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ video_input         │ (None, 32, 64,    │          0 │ -                 │
│ (InputLayer)        │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 32, 64,    │        896 │ video_input[0][0] │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 64,    │          0 │ conv3d[0][0]      │
│ (Activation)        │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 16, 32,    │          0 │ activation[0][0]  │
│ (MaxPooling3D)      │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 16, 32,    │        128 │ max_pooling3d[0]… │
│ (BatchNormalizatio… │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16, 32,    │          0 │ batch_normalizat… │
│                     │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 16, 32,    │     55,360 │ dropout[0][0]     │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 16, 32,    │          0 │ conv3d_1[0][0]    │
│ (Activation)        │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 8, 16, 16, │          0 │ activation_1[0][… │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eeg_input           │ (None, 5, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 2, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 16, 16, │        256 │ max_pooling3d_1[… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 5, 128)    │    328,192 │ eeg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 2, 64)     │    147,712 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 8, 16, 16, │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 5, 128)    │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 2, 64)     │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 131072)    │          0 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 17,499,796 (66.76 MB)

 Trainable params: 17,499,604 (66.76 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 24s 821ms/step - arousal_output_accuracy: 0.1024 - arousal_output_loss: 5.9068 - loss: 12.1713 - valence_output_accuracy: 0.1377 - valence_output_loss: 6.1864 - val_arousal_output_accuracy: 0.1486 - val_arousal_output_loss: 8.2245 - val_loss: 22.1226 - val_valence_output_accuracy: 0.0929 - val_valence_output_loss: 13.9683
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 631ms/step - arousal_output_accuracy: 0.1311 - arousal_output_loss: 2.2833 - loss: 4.5162 - valence_output_accuracy: 0.1822 - valence_output_loss: 2.2304 - val_arousal_output_accuracy: 0.1300 - val_arousal_output_loss: 12.5267 - val_loss: 22.1990 - val_valence_output_accuracy: 0.1022 - val_valence_output_loss: 9.5688
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 636ms/step - arousal_output_accuracy: 0.1411 - arousal_output_loss: 2.2041 - loss: 4.3617 - valence_output_accuracy: 0.1753 - valence_output_loss: 2.1432 - val_arousal_output_accuracy: 0.1269 - val_arousal_output_loss: 7.9804 - val_loss

**Without Duplicate**

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv3D, MaxPooling3D, Flatten, Dense, LSTM, Dropout,
                                   Concatenate, BatchNormalization, Activation)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical

# 1. Data Loading & Duplication
data_path = '/content/drive/MyDrive/dataset/preprocessed-data/mahnob_HCI_preprocessed_all.npy'
data = np.load(data_path, allow_pickle=True)

# 2. Feature Extraction
X_eeg   = np.array([entry['eeg_data'] for entry in data])    # [samples, 5, 512]
X_video = np.array([entry['video_data'] for entry in data])  # [samples, 32, 64, 64]
X_ecg   = np.array([entry['ecg_data'] for entry in data])    # [samples, 2, 512]
y       = np.array([entry['labels'] for entry in data])

print(f"Number of samples : {len(data)}")

# 3. Video Padding (if needed)
if X_video.shape[1:] != (32, 64, 64):
    padded_video = []
    for video in X_video:
        pad_shape = [
            (0, max(0, 32 - video.shape[0])),
            (0, max(0, 64 - video.shape[1])),
            (0, max(0, 64 - video.shape[2]))
        ]
        padded_video.append(np.pad(video, pad_shape, mode='constant'))
    X_video = np.array(padded_video)

# 4. Balance Classes (both valence and arousal)
def balance_classes(X1, X2, X3, y, class_idx):
    labels = y[:, class_idx]
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    max_count = np.max(class_counts)

    balanced_X1, balanced_X2, balanced_X3, balanced_y = [], [], [], []
    for cls in unique_classes:
        cls_mask = labels == cls
        cls_X1, cls_X2, cls_X3, cls_y = X1[cls_mask], X2[cls_mask], X3[cls_mask], y[cls_mask]
        if len(cls_y) < max_count:
            X1_oversampled, X2_oversampled, X3_oversampled, y_oversampled = resample(
                cls_X1, cls_X2, cls_X3, cls_y,
                replace=True,
                n_samples=max_count - len(cls_y),
                random_state=42
            )
            cls_X1 = np.concatenate([cls_X1, X1_oversampled])
            cls_X2 = np.concatenate([cls_X2, X2_oversampled])
            cls_X3 = np.concatenate([cls_X3, X3_oversampled])
            cls_y = np.concatenate([cls_y, y_oversampled])
        balanced_X1.append(cls_X1)
        balanced_X2.append(cls_X2)
        balanced_X3.append(cls_X3)
        balanced_y.append(cls_y)
    return (np.concatenate(balanced_X1),
            np.concatenate(balanced_X2),
            np.concatenate(balanced_X3),
            np.concatenate(balanced_y))

# Balance by valence (index 1) and arousal (index 2)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=1)
X_eeg, X_video, X_ecg, y = balance_classes(X_eeg, X_video, X_ecg, y, class_idx=2)

# 5. Train/Test Split
X_eeg_train, X_eeg_test, X_video_train, X_video_test, X_ecg_train, X_ecg_test, y_train, y_test = train_test_split(
    X_eeg, X_video, X_ecg, y, test_size=0.2, random_state=42
)

num_valence_classes = int(np.max(y[:, 1])) + 1
num_arousal_classes = int(np.max(y[:, 2])) + 1

y_valence_train = to_categorical(y_train[:, 1], num_classes=num_valence_classes)
y_arousal_train = to_categorical(y_train[:, 2], num_classes=num_arousal_classes)
y_valence_test = to_categorical(y_test[:, 1], num_classes=num_valence_classes)
y_arousal_test = to_categorical(y_test[:, 2], num_classes=num_arousal_classes)

# 6. Model Definition
# 6a. State-of-the-Art 3D CNN Video Branch (Kahou et al. style)
video_input = Input(shape=(32, 64, 64, 1), name="video_input")
x = Conv3D(32, (3,3,3), strides=1, padding='same')(video_input)
x = Activation('relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Conv3D(64, (3,3,3), strides=1, padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
cnn_output = Dense(128, activation='relu')(x)

# 6b. EEG Branch (LSTM)
eeg_input = Input(shape=(5, 512), name="eeg_input")
lstm_eeg = LSTM(128, return_sequences=True)(eeg_input)
lstm_eeg = Dropout(0.2)(lstm_eeg)
lstm_eeg = LSTM(64)(lstm_eeg)
lstm_eeg_output = Dense(128, activation='relu')(lstm_eeg)

# 6c. ECG Branch (LSTM)
ecg_input = Input(shape=(2, 512), name="ecg_input")
lstm_ecg = LSTM(64, return_sequences=True)(ecg_input)
lstm_ecg = Dropout(0.2)(lstm_ecg)
lstm_ecg = LSTM(32)(lstm_ecg)
lstm_ecg_output = Dense(64, activation='relu')(lstm_ecg)

# 6d. Fusion & Output
combined = Concatenate()([cnn_output, lstm_eeg_output, lstm_ecg_output])
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.3)(combined)
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.2)(combined)
valence_output = Dense(num_valence_classes, activation='softmax', name="valence_output")(combined)
arousal_output = Dense(num_arousal_classes, activation='softmax', name="arousal_output")(combined)

model = Model(
    inputs=[video_input, eeg_input, ecg_input],
    outputs=[valence_output, arousal_output]
)

model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'categorical_crossentropy'],
    metrics=['accuracy','accuracy']
)

model.summary()

# 7. Data Reshaping for Video Channel
X_video_train_input = X_video_train[..., np.newaxis] if X_video_train.ndim == 4 else X_video_train
X_video_test_input = X_video_test[..., np.newaxis] if X_video_test.ndim == 4 else X_video_test

# 8. Model Training
history = model.fit(
    [X_video_train_input, X_eeg_train, X_ecg_train],
    [y_valence_train, y_arousal_train],
    validation_data=(
        [X_video_test_input, X_eeg_test, X_ecg_test],
        [y_valence_test, y_arousal_test]
    ),
    epochs=50,
    batch_size=128
)

# 9. Model Evaluation
y_valence_pred, y_arousal_pred = model.predict([X_video_test_input, X_eeg_test, X_ecg_test])
y_valence_pred_classes = np.argmax(y_valence_pred, axis=1)
y_arousal_pred_classes = np.argmax(y_arousal_pred, axis=1)
y_valence_true = np.argmax(y_valence_test, axis=1)
y_arousal_true = np.argmax(y_arousal_test, axis=1)

valence_accuracy = accuracy_score(y_valence_true, y_valence_pred_classes)
arousal_accuracy = accuracy_score(y_arousal_true, y_arousal_pred_classes)

print(f"Valence Accuracy: {valence_accuracy * 100:.2f}%")
print(f"Arousal Accuracy: {arousal_accuracy * 100:.2f}%")


Number of samples : 186


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ video_input         │ (None, 32, 64,    │          0 │ -                 │
│ (InputLayer)        │ 64, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 32, 64,    │        896 │ video_input[0][0] │
│                     │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 32, 64,    │          0 │ conv3d_2[0][0]    │
│ (Activation)        │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_2     │ (None, 16, 32,    │          0 │ activation_2[0][… │
│ (MaxPooling3D)      │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 32,    │        128 │ max_pooling3d_2[… │
│ (BatchNormalizatio… │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 16, 32,    │          0 │ batch_normalizat… │
│                     │ 32, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 16, 32,    │     55,360 │ dropout_6[0][0]   │
│                     │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 16, 32,    │          0 │ conv3d_3[0][0]    │
│ (Activation)        │ 32, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_3     │ (None, 8, 16, 16, │          0 │ activation_3[0][… │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eeg_input           │ (None, 5, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ecg_input           │ (None, 2, 512)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 16, 16, │        256 │ max_pooling3d_3[… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 5, 128)    │    328,192 │ eeg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ (None, 2, 64)     │    147,712 │ ecg_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 8, 16, 16, │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 5, 128)    │          0 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 2, 64)     │          0 │ lstm_6[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 131072)    │          0 │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 17,499,796 (66.76 MB)

 Trainable params: 17,499,604 (66.76 MB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - arousal_output_accuracy: 0.0476 - arousal_output_loss: 11.3431 - loss: 20.2230 - valence_output_accuracy: 0.0787 - valence_output_loss: 8.9006 - val_arousal_output_accuracy: 0.1809 - val_arousal_output_loss: 7.8476 - val_loss: 15.3476 - val_valence_output_accuracy: 0.1383 - val_valence_output_loss: 7.5000
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 794ms/step - arousal_output_accuracy: 0.1640 - arousal_output_loss: 6.0443 - loss: 11.7292 - valence_output_accuracy: 0.1968 - valence_output_loss: 5.6548 - val_arousal_output_accuracy: 0.0532 - val_arousal_output_loss: 30.1933 - val_loss: 51.6920 - val_valence_output_accuracy: 0.0638 - val_valence_output_loss: 21.4987
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 807ms/step - arousal_output_accuracy: 0.1433 - arousal_output_loss: 4.1873 - loss: 7.8716 - valence_output_accuracy: 0.1453 - valence_output_loss: 3.6553 - val_arousal_output_accuracy: 0.0745 - val_arousal_output_loss: 21.1672 - val_loss: 36.66